# importing libraries

In [ ]:
import os
import sys
homedir = "../"
codedir = os.path.join(homedir, 'src')
sys.path.append(codedir)
import map_buildings_original, utils, map_buildings_custom

from shapely.geometry import MultiPolygon, Polygon
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np

# classify buildings

In [ ]:
# Load polygon and set output filename
polygon = gpd.read_file('../data/new_orleans_nasa_extent.gpkg').iloc[0].geometry
filename_out = '../data/la_clipped_output_osm.gpkg'

# Select footprint type: 'original' or 'custom'
type_tag = 'original'
if type_tag == 'custom':
    custom_footprint = gpd.read_file('../data/la_microsoft_buildings/la_clipped.gpkg')
    custom_footprint['element'] = 'way'
    custom_footprint['id'] = range(len(custom_footprint))
    custom_footprint['building'] = ''

In [ ]:
# Generate GeoDataFrame with building segments
if type_tag == 'custom':
    gdf_identified, footprint_id2features, gdf_not_used = map_buildings_custom.generate_gdf_with_segments(polygon, 1, map_buildings_custom.tags,custom_footprint)
else:
    gdf_identified, footprint_id2features, gdf_not_used = map_buildings_original.generate_gdf_with_segments(polygon, 1, map_buildings_original.tags)

# Ensure geometries are MultiPolygons and filter out Points and LineStrings
gdf_identified['geometry'] = gdf_identified['geometry'].apply(lambda geom: MultiPolygon([geom]) if isinstance(geom, Polygon) else geom)
gdf_identified = gdf_identified[gdf_identified.geometry.type != 'Point']
gdf_identified = gdf_identified[gdf_identified.geometry.type != 'LineString']

  0%|          | 0/1 [00:00<?, ?it/s]

/opt/anaconda3/envs/geoai/lib/python3.13/site-packages/shapely/set_operations.py:451: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)
/opt/anaconda3/envs/geoai/lib/python3.13/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/geoai/lib/python3.13/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/geoai/lib/python3.13/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/geoai/lib/python3.13/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/geoai/lib/python3.13/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value enco

In [ ]:
# Reproject to UTM
utm_crs = utils.get_utm_crs_from_geodataframe(gdf_identified)
gdf_identified = gdf_identified.to_crs(epsg=utm_crs) 

# Set MultiIndex
if gdf_identified.shape[0] > 0:
    gdf_identified.index = pd.MultiIndex.from_tuples(gdf_identified.index, names=['el_type', 'osmid'])

# Use auxiliary data to classify buildings
if type_tag == 'custom':
    gdf_identified = map_buildings_custom.use_auxiliary_data(gdf_identified, footprint_id2features)
else:
    gdf_identified = map_buildings_original.use_auxiliary_data(gdf_identified, footprint_id2features)

#Add extra col
gdf_identified['no_match'] = gdf_identified['aux info'] == 'residential_unknown_tag'

# Save to file
gdf_identified.to_file(filename_out, driver="GPKG")

/opt/anaconda3/envs/geoai/lib/python3.13/site-packages/shapely/set_operations.py:553: RuntimeWarning: invalid value encountered in unary_union
  return lib.unary_union(collections, **kwargs)
100%|██████████| 323195/323195 [34:31<00:00, 155.99it/s]  
